In [2]:
import csv
import numpy as np
import re
import matplotlib.pyplot as plt

regex1 = r"\d+"
regex2 = r"c\d+"
X = np.zeros((10000,1000)) #1.000x10.000 matrix

In [3]:
#load data
with open("data_train.csv") as csvfile:
	readCSV = csv.reader(csvfile,delimiter=',')
	next(readCSV) #discard header
	for row in readCSV:
		match = re.search(regex1,row[0])
		match2 = re.search(regex2,row[0])
		row_i = int(match.group(0))-1
		col_i = int(match2.group(0)[1:]) -1
		X[row_i,col_i] = int(row[1])

In [4]:
def prediction(U,Z):
    return np.dot(U.T,Z)

In [5]:
M,N = X.shape
K = 100
U = np.random.rand(K,M)
Z = np.random.rand(K,N)
n_epochs = 2
lmda = 0.1
gamma = 0.01


In [6]:
#DO SGD
t = 1
users,movies = X.nonzero()
print(users.shape)
print(movies.shape)


(1176952,)
(1176952,)


In [11]:
for epoch in range(n_epochs):
    for u, i in zip(users,movies):
        e = X[u, i] - prediction(U[:,u],Z[:,i])
        print(X[u,i],prediction(U[:,u],Z[:,i]))
        U[:,u] += gamma * ( e * Z[:,i] - lmda * U[:,u])
        Z[:,i] += gamma * ( e * U[:,u] - lmda * Z[:,i])  

KeyboardInterrupt: 

In [ ]:
#now lets predict the data from samplesubmission
fout = open('mysubmission.csv', 'w')
fout.write("Id,Prediction\n")
with open("SampleSubmission.csv") as csvfile:
    readCSV = csv.reader(csvfile,delimiter=',')
    next(readCSV) #discard header
    for row in readCSV:
        match = re.search(regex1,row[0])
        match2 = re.search(regex2,row[0])
        row_i = int(match.group(0))-1
        col_i = int(match2.group(0)[1:]) -1
        calc = prediction(U[:,row_i], Z[:,col_i])
        print(calc)
        fout.write(row[0]+","+str(calc)+"\n")
    fout.close()